## Radarbanten.co.id Search 

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading
import re

In [2]:
jumlah_index = 1
threads_link = []
links = []
results = []
threads = []
keywords = 'ganjar'

In [3]:
def scrape_links(page_number,keywords):
    url=f"https://www.radarbanten.co.id/page/{page_number}/?s={keywords}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('h3',{"class": "jeg_post_title"})
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [4]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p,keywords)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 18 links from page 1
Total Links: 18


In [5]:
def scrape_url(url,keywords):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')        
            # Judul Berita
            title_elem = soup.find('h1')
            if title_elem:
                title_text = title_elem.text
            else:
                title_text = "Title not found"
            # Author berita
            author_elem = soup.find('div', {"class": "jeg_meta_author"})
            if author_elem:
                author_text = author_elem.find('a')
                author_text = author_text.text
            else:
                author_text = "Author not found"
                     
            # tanggal berita
            date_elem = soup.find('div', {"class": "jeg_meta_date"})
            if date_elem:
                date_text = date_elem.find('a')
                date_text = date_text.text
            else:
                date_text = "Date not found"
            # Category berita
            category_elements = soup.find('span', {"class": "breadcrumb_last_link"})
            if category_elements:
                category_text = category_elements.find('a')
                category_text= category_text.text
            else:
                category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"class": "entry-content with-share"})
            
            if body_elem:
                content_elem = body_elem.find_all('p')
                content_text = ""
                for p in content_elem:
                    content_text += p.text.strip() + "\n"
                
                if content_text.strip():
                    content_text=content_text
                else:
                    content_text ="Content not found"
            else:
                content_text ="Content not found"
            nama_berita_match = re.search(r'https://www\.(\w+)\.co\.id/', url)
            if nama_berita_match:
                nama_berita = nama_berita_match.group(1)
            else:
                nama_berita = "Nama_berita not found"


            results.append({'title': title_text,
                            'keywords':keywords,
                            'author' : author_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                            'nama_berita' : nama_berita,
                            'region':'banten',
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [6]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,keywords))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [7]:
df = pd.DataFrame(results)
print("Total Hasil ",len(results))
df.head(10)

Total Hasil  18


,title,keywords,author,category,date,content,nama_berita,region,link
0,"Selain Bayam, Inilah 9 Sayuran yang Bahaya Jik...",ganjar,Siti Fatimah Azzahro,Kesehatan,04-05-2023 14:17:09,RADARBANTEN.CO.ID – Sayuran memang menyehatkan...,radarbanten,banten,https://www.radarbanten.co.id/selain-bayam-ini...
1,Pospera Gelar Konsolidasi Bahas Pemenangan Gan...,ganjar,Angger Gita,Berita Utama,04-09-2023 09:47:58,"TANGERANG,RADARBANTEN.CO.ID-Konsolidasi kader ...",radarbanten,banten,https://www.radarbanten.co.id/pospera-gelar-ko...
2,Distan Banten Ungkap Penyebab Tingginya Harga ...,ganjar,Yusuf Permana,Utama,06-10-2023 13:23:41,"SERANG, RADARBANTEN.CO.ID – Dinas Pertanian (D...",radarbanten,banten,https://www.radarbanten.co.id/distan-banten-un...
3,"Krisis Air Bersih di Kasemen, Bantuan Terus Me...",ganjar,Eko Fajar,Kota Serang,06-10-2023 13:49:43,RADARBANTEN.CO.ID – Krisis air bersih di Kasem...,radarbanten,banten,https://www.radarbanten.co.id/krisis-air-bersi...
4,"Didukung SBY, Prabowo Menang Telak dari Anies ...",ganjar,Yusuf Permana,Politik,18-09-2023 17:59:16,"SERANG, RADARBANTEN.CO.ID – Koalisi Indonesia ...",radarbanten,banten,https://www.radarbanten.co.id/didukung-sby-pra...
5,Polda Sebut Keterlibatan Mantan Ketua DPRD Cil...,ganjar,Fahmi,Utama,04-10-2023 14:48:23,"SERANG, RADARBANTEN.CO.ID – Mantan Ketua DPRD ...",radarbanten,banten,https://www.radarbanten.co.id/polda-sebut-kete...
6,PDIP Buka Peluang Demokrat Berkoalisi Menangka...,ganjar,Yusuf Permana,Berita Utama,11-09-2023 11:19:22,"SERANG,RADARBANTEN.CO.ID-Partai Demokrasi Indo...",radarbanten,banten,https://www.radarbanten.co.id/pdip-buka-peluan...
7,"Popularitas Ganjar di Banten Masih Rendah, Has...",ganjar,Yusuf Permana,Politik,10-09-2023 16:05:02,"SERANG, RADARBANTEN.CO.ID – Popularitas dari b...",radarbanten,banten,https://www.radarbanten.co.id/popularitas-ganj...
8,"Dugaan Korupsi Pasar Grogol, Asda II Cilegon S...",ganjar,Fahmi,Berita Utama,03-10-2023 09:51:12,"SERANG,RADARBANTEN.CO.ID-Asda II Kota Cilegon ...",radarbanten,banten,https://www.radarbanten.co.id/dugaan-korupsi-p...
9,"Gawat, Relawan Prabowo Saat Pilpres 2019 Ini B...",ganjar,Mulyadi,Politik,15-09-2023 13:12:09,"TANGERANG , RADARBANTEN.CO. ID – Relawan Prabo...",radarbanten,banten,https://www.radarbanten.co.id/gawat-relawan-pr...


In [8]:
filtered_df = df[df['content'].str.contains(keywords, case=False)|
                 df['title'].str.contains(keywords, case=False)]
print('hasil filter',len(filtered_df))
filtered_df.head(10)

hasil filter 10


,title,keywords,author,category,date,content,nama_berita,region,link
1,Pospera Gelar Konsolidasi Bahas Pemenangan Gan...,ganjar,Angger Gita,Berita Utama,04-09-2023 09:47:58,"TANGERANG,RADARBANTEN.CO.ID-Konsolidasi kader ...",radarbanten,banten,https://www.radarbanten.co.id/pospera-gelar-ko...
4,"Didukung SBY, Prabowo Menang Telak dari Anies ...",ganjar,Yusuf Permana,Politik,18-09-2023 17:59:16,"SERANG, RADARBANTEN.CO.ID – Koalisi Indonesia ...",radarbanten,banten,https://www.radarbanten.co.id/didukung-sby-pra...
6,PDIP Buka Peluang Demokrat Berkoalisi Menangka...,ganjar,Yusuf Permana,Berita Utama,11-09-2023 11:19:22,"SERANG,RADARBANTEN.CO.ID-Partai Demokrasi Indo...",radarbanten,banten,https://www.radarbanten.co.id/pdip-buka-peluan...
7,"Popularitas Ganjar di Banten Masih Rendah, Has...",ganjar,Yusuf Permana,Politik,10-09-2023 16:05:02,"SERANG, RADARBANTEN.CO.ID – Popularitas dari b...",radarbanten,banten,https://www.radarbanten.co.id/popularitas-ganj...
9,"Gawat, Relawan Prabowo Saat Pilpres 2019 Ini B...",ganjar,Mulyadi,Politik,15-09-2023 13:12:09,"TANGERANG , RADARBANTEN.CO. ID – Relawan Prabo...",radarbanten,banten,https://www.radarbanten.co.id/gawat-relawan-pr...
10,Even Basket Ganjar Challenge: Clash of The Tit...,ganjar,Angger Gita,Utama,31-08-2023 12:25:45,"TANGERANG, RADARBANTEN.CO.ID- Persatuan Bola B...",radarbanten,banten,https://www.radarbanten.co.id/even-basket-ganj...
12,"Pemkot Cilegon Diganjar Penghargaan, Dianggap ...",ganjar,Bayu Mulyana,Utama,04-10-2023 13:32:56,"CILEGON,RADARBANTEN.CO.ID – Pemkot Cilegon ber...",radarbanten,banten,https://www.radarbanten.co.id/pemkot-cilegon-d...
13,Ganjar Disukai Warga Banten Tapi Popularitasny...,ganjar,Aas Arbi,Politik,10-09-2023 14:54:33,"SERANG, RADARBANTEN.CO.ID – PDIP menyebut baka...",radarbanten,banten,https://www.radarbanten.co.id/ganjar-disukai-w...
14,Survey Elektabilitas Ganjar-Ridwan Kamil di Ba...,ganjar,Angger Gita,Berita Utama,13-09-2023 17:56:06,"\nTANGERANG, RADARBANTEN.CO.ID – Tingginya ele...",radarbanten,banten,https://www.radarbanten.co.id/survey-elektabil...
17,Sekjen PDIP Sebut Banten Sangat Penting Bagi G...,ganjar,Yusuf Permana,Politik,10-09-2023 14:15:50,"SERANG, RADARBANTEN.CO.ID – Sekretaris Jendera...",radarbanten,banten,https://www.radarbanten.co.id/sekjen-pdip-sebu...


# Save File 

In [9]:
# current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# excel_file_name = f'../../tempat_hasil_daerah/radarBanten_{keywords}_{current_datetime}.xlsx'
# filtered_df.to_excel(excel_file_name, index=False)

# print(f'Data has been saved to {excel_file_name}')